Once I got annoyed, that I need to click so much on the Airport website to see flights, so I found this to be a great opportunity to play around with code a little.

In [ ]:
import numpy as np
import pandas as pd
import requests
import time
import urllib

In [ ]:
# BeautifulSoup is only able to deal with static websites.
from bs4 import BeautifulSoup

# Selenium can handle dynamic websites. You know a site is dynamic, because Soup will not show the code, since it is called later
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # or Firefox

year = 2025
month = int(input('Which month you want to travel? With number please'))
day = int(input('Which day you want to travel? With number please'))
hour = int(input('From which hour should we check? Just a number please.'))

if hour < 10:
    hour = f'0{hour}'  #if the hour is one digit, eg. 6, we have to convert to 06
if day < 10:
    day = f'0{day}'
if month < 10:
    month = f'0{month}'

url = (f'https://www.cph.dk/en/flight-information/departures?date={day}%20%2F%20{month}%20%2F%20{year}&time={hour}')
print(url)

driver.get(url)

# handling the cookie popup
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "coi-banner__accept")) 
    )
    accept_button.click()
except:
    print("No cookie popup or couldn't find it.")
    

#how much times shall we click the "load more" button
for i in range(22):
    try:
        show_more = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button--expander"))
        )
        show_more.click()
        time.sleep(1.5)  # wait for content to load
    except:
        print(f"Stopped after {i} clicks — error or no more results.")
        break


import time
time.sleep(5)

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()


https://www.cph.dk/en/flight-information/departures?date=28%20%2F%2010%20%2F%202025&time=08


Find elements on website

In [ ]:
times = soup.find_all('em') # departure times
times_as_text =  [i.get_text(strip=True) for i in times]
print(len(times_as_text))
print(times_as_text)

230
['08:00', '08:00', '08:00', '08:00', '08:05', '08:05', '08:05', '08:05', '08:05', '08:10', '08:10', '08:10', '08:15', '08:15', '08:15', '08:15', '08:15', '08:20', '08:20', '08:20', '08:20', '08:25', '08:25', '08:25', '08:30', '08:30', '08:30', '08:30', '08:30', '08:35', '08:35', '08:35', '08:35', '08:35', '08:35', '08:40', '08:40', '08:45', '08:45', '08:50', '08:55', '08:55', '09:00', '09:00', '09:05', '09:05', '09:15', '09:30', '09:30', '09:35', '09:40', '09:45', '09:50', '09:50', '09:50', '09:55', '10:00', '10:00', '10:00', '10:00', '10:10', '10:10', '10:15', '10:20', '10:20', '10:30', '10:40', '10:45', '10:45', '10:45', '10:55', '10:55', '11:00', '11:00', '11:00', '11:15', '11:20', '11:20', '11:20', '11:25', '11:25', '11:45', '11:55', '12:00', '12:05', '12:10', '12:15', '12:20', '12:25', '12:25', '12:30', '12:40', '12:40', '12:40', '12:40', '12:45', '12:45', '12:45', '12:45', '12:45', '12:45', '12:50', '12:50', '12:50', '12:50', '12:55', '12:55', '12:55', '12:55', '12:55', '13:0

In [ ]:
destinations = soup.find_all('div', class_='stylish-table__cell flights__table__col--destination')  # destinations
destinations_text = [d.get_text(strip=True) for d in destinations]


elements =list()
for d in destinations:
    parts = list(d.stripped_strings)
    elements.append(parts)
    

Put elements in a dictionary

In [10]:
flights = []
for d in destinations:
    parts = list(d.stripped_strings)
    if len(parts) >= 4:
        flights.append({
            'destination': parts[0],
            'flight_code': parts[2],
            'airline': parts[3]
        })

Compose it to a pandas dataframe

In [ ]:
Departures = pd.DataFrame(
    data=[[t] + e for t, e in zip(times_as_text, elements)],
    columns=['Time', 'Destination', 'Dest', 'IATA', 'Flight_Nr', 'Airline']
)

Departures = Departures.drop(['Dest'], axis=1)
Departures["IATA"] = Departures["IATA"].str.replace(r"[()\[\]{}]", "", regex=True)

Departures

,Time,Destination,IATA,Flight_Nr,Airline
0,08:00,Stockholm/ARN,ARN,SK1416,Scandinavian Airlines
1,08:00,Brussels,BRU,SK593,Scandinavian Airlines
2,08:00,Helsinki,HEL,SK1706,Scandinavian Airlines
3,08:00,Trondheim,TRD,SK2882,Scandinavian Airlines
4,08:05,Stuttgart,STR,SK657,Scandinavian Airlines
...,...,...,...,...,...
225,19:00,Stockholm/ARN,ARN,D84168,Norwegian Air Sweden
226,19:00,Frankfurt,FRA,LH831,Lufthansa
227,19:00,Stockholm/ARN,ARN,SK1430,Scandinavian Airlines
228,19:10,Bologna,BLQ,FR2677,Ryanair


In [ ]:
Departures.to_csv("Filename.csv")

For additional airport information (if someone is curious), one can always use the IATA Airports API (available on RapidApi)

Requests are based on IATA codes.

In [ ]:
airport_codes = set(Departures.iloc[:,2])
print(len(airport_codes))

airport_details = []
for code in airport_codes:
  url = f"https://iata-airports.p.rapidapi.com/airports/{code}/"
  
  headers = {
	"x-rapidapi-key": "06e35e1965mshdeeab512c60c0f6p1559e5jsn6784978fd991",
	"x-rapidapi-host": "iata-airports.p.rapidapi.com"
  }
  response = requests.get(url, headers=headers)
  airport_details.append(response.json())


86


In [27]:
Airports = pd.DataFrame.from_records(airport_details, index=None)
Airports.head(10)

,code,icao,name,latitude,longitude,elevation,url,time_zone,city_code,country,city,state,county,type
0,BKK,VTBS,Suvarnabhumi Airport,13.681877,100.748580,29,None,Asia/Bangkok,BKK,TH,Lat Krabang,Bangkok,None,AP
1,BUD,LHBP,Budapest Ferenc Liszt International Airport,47.435273,19.253511,469,http://www.bud.hu/english,Europe/Budapest,BUD,HU,Vecses,Pest,None,AP
2,AMS,EHAM,Amsterdam Airport Schiphol,52.326979,4.741505,36,http://www.schiphol.nl/,Europe/Amsterdam,AMS,NL,Aalsmeer,North Holland,Gemeente Aalsmeer,AP
3,VIE,LOWW,Vienna International Airport,48.104997,16.584899,561,http://www.viennaairport.com/en/,Europe/Vienna,VIE,AT,Fischamend Dorf,Lower Austria,Politischer Bezirk Wien-Umgebung,AP
4,ARN,ESSA,Stockholm Arlanda Airport,59.646792,17.937044,88,http://www.swedavia.se/arlanda/,Europe/Stockholm,STO,SE,Marsta,Stockholm,Sigtuna Kommun,AP
5,BCN,LEBL,Barcelona-El Prat Airport,41.296944,2.079047,22,http://www.aena.es/en/barcelona-airport/index....,Europe/Madrid,BCN,ES,El Prat de Llobregat,Catalonia,Provincia de Barcelona,AP
6,LIS,LPPT,Lisbon Portela Airport,38.780954,-9.131901,374,http://www.ana.pt/en-US/Aeroportos/lisboa/Lisb...,Europe/Lisbon,LIS,PT,Moscavide,Lisbon,Loures,AP
7,DXB,OMDB,Dubai Airport,25.248665,55.352917,39,http://www.dubaiairport.com/dia/english/home/,Asia/Dubai,DXB,AE,None,None,None,AP
8,KEF,BIKF,Keflavik International Airport,63.984861,-22.626598,171,https://www.isavia.is/en/keflavik-airport,Atlantic/Reykjavik,REK,IS,Reykjanesbaer,Southern Peninsula,None,AP
9,BLL,EKBI,Billund Airport,55.740737,9.160392,219,http://www.billund-airport.dk/?sc_lang=en,Europe/Copenhagen,BLL,DK,Billund,South Denmark,Billund Kommune,AP
